In [1]:
from IPython.display import display
from cobramod import __version__
from cobra import __version__ as cobra_version
print(f'CobraMod version: {__version__}')
print(f'COBRApy version: {cobra_version}')
# From Escher:
# This option turns off the warning message if you leave or refresh this page
import escher
escher.rc['never_ask_before_quit'] = True

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
CobraMod version: 0.5.5-alpha.1
COBRApy version: 0.22.1



# Analyising shikimate production using an *E. coli* core model

## Introduction

In this tutorial we use a core model of [*E. coli*](
http://bigg.ucsd.edu/models/e_coli_core
) [2] and extend it with [shikimate synthesis pathway](
  https://biocyc.org/ECOLI/NEW-IMAGE?type=PATHWAY&object=ARO-PWY
). Shikimate is an important precursor for aromatic compounds such as
phenylalanine, tyrosine, and tryptophan. In [this article](
https://doi.org/10.1016/j.biortech.2014.05.035) the authors *in vivo*
engineered five *E. coli* strains for shikimate overproduction by deactivating
genes for the enzymatic regulation of the shikimate metabolism [1].

In our test case, we compare the control strain (`W3110`) with a strain (`SA3`)
from the article. We use CobraMod to add the shikimate pathway from the BioCyc
sub-database [EcoCyc](
 https://www.ecocyc.org/
) for metabolic pathway information of *E. coli*. We then simulate and
visualize shikimate production for the two strains. The shikimate pathway is shown below and is represented by
two sub-pathways with the EcoCyc identifiers [PWY-6164](
  https://biocyc.org/ECOLI/NEW-IMAGE?type=PATHWAY&object=PWY-6164
) and [PWY-6163](
  https://biocyc.org/ECOLI/NEW-IMAGE?type=PATHWAY&object=PWY-6163
). 

<img src="https://websvc.biocyc.org/ECOLI/diagram-only?type=PATHWAY&object=ARO-PWY&pfontsize=normal&linear=snake"/>


<table style="border-collapse: collapse; width: 100%;" border="1">
<tbody>
<tr>
<td style="width: 33.3333%;"><strong>Pathway identifier</strong></td>
<td style="width: 33.3333%;"><strong>Pathway name</strong></td>
</tr>
<tr>
<td style="width: 33.3333%;">PWY-6164</td>
<td style="width: 33.3333%;">3-dehydroquinate biosynthesis I</td>
</tr>
<tr>
<td style="width: 33.3333%;">PWY-6163</td>
<td style="width: 33.3333%;">chorismate biosynthesis from 3-dehydroquinate</td>
</tr>
</tbody>
</table>




## Loading CobraMod and preparing medium for the simulation

First, we use the *E. coli* core model from COBRApy as our model. This core model can be found under `cobramod.test.textbook` This core model contains 72
metabolites, 95 reactions and 137 genes. We load the function
[cobramod.add_pathway()](
module/cobramod/index.html#cobramod.add_pathway
), and the Python module [pathlib](
https://docs.python.org/3/library/pathlib.html
) to represent the system path where CobraMod stores the metabolic pathway
information. We define the directory for pathway information as `dir_data`. The
pathways with the respective reaction, gene and metabolite information are 
saved here.

In [2]:
from pathlib import Path

from cobramod import add_pathway
from cobramod.test import textbook

# Defining system path for data
dir_data = Path.cwd().joinpath("data")
# Defining model
model = textbook.copy()

In the selected study, the authors used a culture medium including glucose and several other metabolites. Since only glucose is included in our core model and for the sake of simplicity we only constrain glucose and oxygen uptake.

In [3]:
# limiting glucose
model.exchanges.EX_glc__D_e.bounds = ( -10, 0)
# limiting oxygen
model.exchanges.EX_o2_e.bounds = (0, 0)

## Adding the shikimate pathways

To add a pathway to the model we use the function [add_pathway()](
module/cobramod/index.html#cobramod.add_pathway
). To do so, the first argument `model` is the respective model to extend. The
argument `pathway` is the respective database-specific identifier for the pathway (`PWY-6164` and `PWY-6163`) following the database identifier `ECOLI`.
We set the cytosol (`c`) as the compartment for the reactions and metabolites
of the pathways. Because those sub-pathways are part of a larger pathway, we use
the argument `group` to set create a single [cobramod.Pathway](
module/cobramod/index.html#cobramod.Pathway
) instead of two and merge both pathways.

CobraMod tries avoiding duplicates reactions and metabolites. In our case, the
`textbook` model uses BiGG identifiers and the downloaded metabolic pathway information use EcoCyc identifiers. When downloading data CobraMod checks the
cross-references within the pathway information and searches for corresponding objects in the model. In this example CobraMod warns the user that metabolites
such as `Pi` (phosphate), `WATER` and `ATP` are already included in the model.

Lastly, calling the pathway object shows a table with its important attributes
such as identifier, reactions and genes involved.

In [4]:
# first pathway
add_pathway(
    model=model,
    pathway="PWY-6164",
    database="ECOLI",
    compartment="c",
    group="PWY-SHIKIMATE",
    directory=dir_data,
)

# second pathway
add_pathway(
    model=model,
    pathway="PWY-6163",
    database="ECOLI",
    compartment="c",
    group="PWY-SHIKIMATE",
    directory=dir_data,
)

# Show attributes
model.groups.get_by_id("PWY-SHIKIMATE")

/home/stefano/Documents/cobramod/src/cobramod/core/creation.py:161: UserWarning: Metabolite 'PHOSPHO-ENOL-PYRUVATE' was found as 'pep_c'. Please curate if necessary.
  warn(message=msg, category=UserWarning)
/home/stefano/Documents/cobramod/src/cobramod/core/creation.py:161: UserWarning: Metabolite 'ERYTHROSE-4P' was found as 'e4p_c'. Please curate if necessary.
  warn(message=msg, category=UserWarning)
/home/stefano/Documents/cobramod/src/cobramod/core/creation.py:161: UserWarning: Metabolite 'WATER' was found as 'h2o_c'. Please curate if necessary.
  warn(message=msg, category=UserWarning)
/home/stefano/Documents/cobramod/src/cobramod/core/creation.py:161: UserWarning: Metabolite 'Pi' was found as 'pi_c'. Please curate if necessary.
  warn(message=msg, category=UserWarning)
/home/stefano/Documents/cobramod/src/cobramod/core/extension.py:573: UserWarning: Auxiliary sink reaction for "SK_DEHYDROQUINATE_c" created. Consider removing it and adding the synthesis reactions for the metaboli

Number of       new   | removed entities in
*=====================|===================*
Reactions        2    |    0              
Metabolites      2    |    0              
Exchange         0    |    0              
Demand           0    |    0              
Sinks            1    |    0              
Genes            4    |    0              
Groups           1    |    0              



/home/stefano/Documents/cobramod/src/cobramod/core/creation.py:161: UserWarning: Metabolite 'NADP' was found as 'nadp_c'. Please curate if necessary.
  warn(message=msg, category=UserWarning)
/home/stefano/Documents/cobramod/src/cobramod/core/creation.py:161: UserWarning: Metabolite 'PROTON' was found as 'h_c'. Please curate if necessary.
  warn(message=msg, category=UserWarning)
/home/stefano/Documents/cobramod/src/cobramod/core/creation.py:161: UserWarning: Metabolite 'NADPH' was found as 'nadph_c'. Please curate if necessary.
  warn(message=msg, category=UserWarning)
/home/stefano/Documents/cobramod/src/cobramod/core/creation.py:161: UserWarning: Metabolite 'ATP' was found as 'atp_c'. Please curate if necessary.
  warn(message=msg, category=UserWarning)
/home/stefano/Documents/cobramod/src/cobramod/core/creation.py:161: UserWarning: Metabolite 'ADP' was found as 'adp_c'. Please curate if necessary.
  warn(message=msg, category=UserWarning)


Number of       new   | removed entities in
*=====================|===================*
Reactions        5    |    0              
Metabolites      5    |    0              
Exchange         0    |    0              
Demand           0    |    0              
Sinks            1    |    1              
Genes            6    |    0              
Groups           0    |    0              



/home/stefano/Documents/cobramod/src/cobramod/core/extension.py:573: UserWarning: Auxiliary sink reaction for "SK_CHORISMATE_c" created. Consider removing it and adding the synthesis reactions for the metabolite.
  warn(message=msg, category=UserWarning)


Pathway identifier,PWY-SHIKIMATE
Name,
Memory address,0x0139847409416080
Reactions involved,"DAHPSYN_RXN_c, 3_DEHYDROQUINATE_SYNTHASE_RXN_c, 3_DEHYDROQUINATE_DEHYDRATASE_RXN_c, SHIKIMATE_5_DEHYDROGENASE_RXN_c, SHIKIMATE_KINASE_RXN_c, 2.5.1.19_RXN_c, CHORISMATE_SYNTHASE_RXN_c"
Genes involved,"EG10080, EG10079, EG10078, EG10074, EG10076, EG10077, EG10082, EG10081, EG10073, EG10075"
Visualization attributes,vertical = False color_negative = None color_positive = None color_quantile = False


Currently, CobraMod cannot visually merge sub-pathways. The first time we call
the function [Pathway.visualize()](
module/cobramod/core/pathway/index.html#cobramod.core.pathway.Pathway.visualize
), CobraMod shows both pathways one above the other, instead of a single lineal
pathway. To fix this, we use the method [Pathway.modify_graph()](
module/cobramod/core/pathway/index.html#cobramod.core.pathway.Pathway.modify_graph
). This function modifies the pathway by connecting two reactions in the 
visualization. The argument `reaction` is a reaction which does not have
subsequent reaction. The argument `next_reaction` sets the subsequent new reaction.

In the first visualization, `3_DEHYDROQUINATE_SYNTHASE_RXN_c` and
`3_DEHYDROQUINATE_DEHYDRATASE_RXN_c` are not connected in the pathway. We use
the `modify_graph` method to join both reactions. The dehydratase reaction is
the subsequent reaction (`next_reaction`) of the synthase (`reaction`). When we
visualize the pathway, it shows now a lineal pathway.

In [5]:
# Show pathway map
display(model.groups.get_by_id("PWY-SHIKIMATE").visualize())
# Merge reactions of the sub-pathways
model.groups.get_by_id("PWY-SHIKIMATE").modify_graph(
    reaction="3_DEHYDROQUINATE_SYNTHASE_RXN_c",
    next_reaction="3_DEHYDROQUINATE_DEHYDRATASE_RXN_c"
)
# Show pathway map
model.groups.get_by_id("PWY-SHIKIMATE").visualize()

Builder(never_ask_before_quit=True, reaction_scale={}, reaction_styles=['color', 'text'])

Builder(never_ask_before_quit=True, reaction_scale={}, reaction_styles=['color', 'text'])

## Creating biomass reactions for the two *E. coli* strains

Table 2 of the article shows the biomass and shikimate production for the different *E. coli* strains tested.  For the two strains of interest, these are following values:

<table style="border-collapse: collapse; width: 100%;" border="1">
<tbody>
<tr>
<td style="width: 33.3333%;"><strong>Strains</strong></td>
<td style="width: 33.3333%;"><strong>Biomass (g/L)</strong></td>
<td style="width: 33.3333%;"><strong>Shikimate (mg/L)</strong></td>
</tr>
<tr>
<td style="width: 33.3333%;">W3110</td>
<td style="width: 33.3333%;">3.42 &plusmn; 0.26</td>
<td style="width: 33.3333%;">1.31 &plusmn; 0.12</td>
</tr>
<tr>
<td style="width: 33.3333%;">SA3</td>
<td style="width: 33.3333%;">5.24 &plusmn; 0.34</td>
<td style="width: 33.3333%;">417.20 &plusmn; 50.01</td>
</tr>
</tbody>
</table>

We use this data to update the biomass equation of our core model by including the contribution of shikimate to the overall biomass composition. For the sake of our tutorial we make some simplifying assumptions (e.g. the ratios of the other biomass components do not change, shikimate productions is coupled to growth) and use the following relationship:


In [6]:
# Calculate ratios for the strains
shikimate_ratio = {
    "W3110": (1.31 / 1000) / 3.42,
    "SA3": (417.20 / 1000) / 5.24
}
shikimate_ratio

{'W3110': 0.00038304093567251464, 'SA3': 0.07961832061068702}

The next step is to calculate the new coefficients for the biomass reactions 
that includes shikimate. The table below shows a dataframe with the metabolite
identifiers, their formula, molecular weight, their biomass reaction
coefficient  and the multiplication between the coefficients and their weights.
Metabolites with negative coefficient values are the reactants, while positive
coefficient values are products in the biomass reaction.

We assume and set the constrain that the sum of the negative coefficients cannot
change and shikimate have a specific proportion (`shikimate_ratio`) in this sum.

In [7]:
import pandas as pd

# Building dataframe from Biomass
biomass_df = pd.DataFrame(
    [metabolite.id, metabolite.formula, metabolite.formula_weight, value]
    for metabolite, value in model.reactions.get_by_id(
        "Biomass_Ecoli_core"
    ).metabolites.items()
)
biomass_df.columns = ["identifier", "formula", "mol_weight", "coefficient"]
biomass_df["mol_weight*coefficient"] = (
    biomass_df["mol_weight"] * biomass_df["coefficient"]
)
# display(biomass_df)

# Calculate Sum of coefficients multiplied by constants
sum_mol_coefficient_negative = biomass_df["mol_weight*coefficient"][
    biomass_df["mol_weight*coefficient"] < 0
].sum()
sum_mol_coefficient_positive = biomass_df["mol_weight*coefficient"][
    biomass_df["mol_weight*coefficient"] > 0
].sum()
print(
    "Sum of molecular weight multiplied by negative coefficients: "
    + f"{sum_mol_coefficient_negative}"
)
print(
    "Sum of molecular weight multiplied by positive coefficients: "
    + f"{sum_mol_coefficient_positive}"
)

Sum of molecular weight multiplied by negative coefficients: -48165.2120532678
Sum of molecular weight multiplied by positive coefficients: 46624.4458868131


We created a function to return a table with the new negative coefficients of 
the biomass reaction. In the first part we calculate the proportion of
shikimate (`shikimate_mol_weight_coefficient`). Then we calculate the proportion
for the rest of the metabolites. Finally, we can divided the new proportions
of each metabolite by its molecular weight. This results in the respective
new coefficient.

The two tables below shows the new coefficient (`new_coefficient`) for each 
strain.

In [8]:
def new_coefficient(ratio: float):
    """
    Returns the a dataframe with the new coefficient for reactants of the biomass
    equation.
    """
    SHIKIMATE_MOL_WEIGHT = 173.14
    shikimate_mol_weight_coefficient = sum_mol_coefficient_negative * ratio
    # Calculating new multiplication between the coefficients and constants
    new_mol_weight_coefficient = biomass_df["mol_weight*coefficient"][
        biomass_df["mol_weight*coefficient"] < 0
    ] * (1 - ratio)
    new_mol_weight_coefficient[
        len(new_mol_weight_coefficient)
    ] = shikimate_mol_weight_coefficient
    # Verify that the sums are the same
    assert round(new_mol_weight_coefficient.sum(), 4) == round(
        sum_mol_coefficient_negative, 4
    )

    # Creating series with names of metabolites
    identifiers = biomass_df["identifier"][
        biomass_df["mol_weight*coefficient"] < 0
    ]
    identifiers[len(identifiers)] = "SHIKIMATE_c"

    # Creating series with molecular weight of metabolites
    mol_weight = biomass_df["mol_weight"][
        biomass_df["mol_weight*coefficient"] < 0
    ]
    mol_weight[len(mol_weight)] = SHIKIMATE_MOL_WEIGHT

    # Building new dataframe
    new_coefficient_df = pd.DataFrame(
        [identifiers, new_mol_weight_coefficient]
    ).transpose()
    new_coefficient_df["new_coefficient"] = new_coefficient_df[
        "mol_weight*coefficient"
    ].divide(mol_weight)

    # Appending old coefficients
    new_coefficient_df["old_coefficient"] = biomass_df["coefficient"][
        biomass_df["mol_weight*coefficient"] < 0
    ]
    return new_coefficient_df

W3110_df_negative = new_coefficient(ratio=shikimate_ratio["W3110"])
SA3_df_negative = new_coefficient(ratio=shikimate_ratio["SA3"])

# Display and format only relevant information
print("Table with new negative coefficients for W3110 strain")
display(W3110_df_negative.loc[:,["identifier","new_coefficient"]])
print("Table with new negative coefficients for SA3 strain")
SA3_df_negative.loc[:,["identifier","new_coefficient"]]

Table with new negative coefficients for W3110 strain


,identifier,new_coefficient
0,3pg_c,-1.495427
1,accoa_c,-3.746364
2,atp_c,-59.78709
3,e4p_c,-0.360862
4,f6p_c,-0.070873
5,g3p_c,-0.128951
6,g6p_c,-0.204921
7,gln__L_c,-0.255602
8,glu__L_c,-4.939507
9,h2o_c,-59.78709


Table with new negative coefficients for SA3 strain


,identifier,new_coefficient
0,3pg_c,-1.376891
1,accoa_c,-3.449406
2,atp_c,-55.048028
3,e4p_c,-0.332258
4,f6p_c,-0.065255
5,g3p_c,-0.118729
6,g6p_c,-0.188678
7,gln__L_c,-0.235342
8,glu__L_c,-4.547974
9,h2o_c,-55.048028


We also calculate the new positive coefficients for the biomass reactions. In 
the biomass equation, there are metabolites that share the same value, for 
example `adp_c` and  `atp_c` with 59.8100. We set the constraint that the sum
of the molecular weight multiply by the coefficient cannot change. We use the 
new coefficients from the previous tables to set the new positive coefficient
for the products.

We created another function to calculate the new positive coefficients. The 
first step is to define the new coefficients of the metabolite that share 
similar values with their counterparts (e.g `adp_c` and `atp_c`). The second
step is to calculate the coefficient of the only metabolite that does not
have a counterpart (`akg_c`).

The tables below shows the new positive coefficients for both strains.

In [9]:
def new_positive_coefficient(df: pd.DataFrame):
    """
    Returns a DataFrame with the new coefficients for the products of the
    biomass reaction. The input dataframe is the dataframe with the new
    coefficients for the reactants.
    """
    # Copy original biomass df and remove columns
    products = biomass_df[biomass_df["mol_weight*coefficient"] > 0].copy()
    del products["formula"]
    del products["mol_weight*coefficient"]
    products.columns = products.columns.str.replace(
        "coefficient", "old_coefficient"
    )   

    # Create new column with the equivalent new coefficients
    products["new_coefficient"] = [
        df["new_coefficient"][2],
        0,
        df["new_coefficient"][1],
        df["new_coefficient"][2],
        df["new_coefficient"][10],
        df["new_coefficient"][11],
        df["new_coefficient"][2],
    ]
    # Update index and convert to absolute value
    products.index = range(0, len(products["new_coefficient"]))
    products["new_coefficient"] = products["new_coefficient"] * -1
    products["mol_weight_coefficient"] = (
        products["mol_weight"].multiply(products["new_coefficient"])
    )

    # Calculate mol weight coefficient of akg. This is the rest from the
    # original sum (sum_mol_coefficient_positive)
    akg_mol_weight_coefficient = (
        sum_mol_coefficient_positive - products["mol_weight_coefficient"].sum()
    )
    # Define new value of akg_mol_weight_coefficient
    products.loc[1, "mol_weight_coefficient"] = akg_mol_weight_coefficient
    
    # The sum of column must be the same as sum_mol_coefficient_positive
    assert round(products["mol_weight_coefficient"].sum(), 4) == round(
        sum_mol_coefficient_positive, 4
    )

    # Calculate back the coefficient of akg
    products.loc[1, "new_coefficient"] = (
        products["mol_weight_coefficient"][1] / products["mol_weight"][1]
    )
    return products
W3110_df_positive = new_positive_coefficient(W3110_df_negative)
SA3_df_positive = new_positive_coefficient(SA3_df_negative)

# Display and format only relevant information
print("Table with new positive coefficients for W3110 strain")
display(W3110_df_positive.loc[:,["identifier","new_coefficient"]])
print("Table with new positive coefficients for SA3 strain")
display(SA3_df_positive.loc[:,["identifier","new_coefficient"]])

Table with new positive coefficients for W3110 strain


,identifier,new_coefficient
0,adp_c,59.787090
1,akg_c,4.240573
2,coa_c,3.746364
3,h_c,59.787090
4,nadh_c,3.545641
5,nadp_c,13.022910
6,pi_c,59.787090


Table with new positive coefficients for SA3 strain


,identifier,new_coefficient
0,adp_c,55.048028
1,akg_c,29.554487
2,coa_c,3.449406
3,h_c,55.048028
4,nadh_c,3.264594
5,nadp_c,11.990640
6,pi_c,55.048028


Finally, we create a function that uses the tables of the reactants and 
products and builds a COBRAPy reaction using their coefficients. After creating
the reactions we added to the core model.

In [10]:
from cobra import Reaction


def create_biomass(
    df_reactants: pd.DataFrame, df_products: pd.DataFrame, identifier: str
):
    """
    Creates a Reaction from the reactant and product DataFrame with new
    coefficients.
    """
    reaction = Reaction(id=identifier)
    # Iterate over reactants
    for index, row in df_reactants.iterrows():
        reaction.add_metabolites(
            {
                model.metabolites.get_by_id(row["identifier"]): row[
                    "new_coefficient"
                ]
            }
        )
    # Iterate over products
    for index, row in df_products.iterrows():
        reaction.add_metabolites(
            {
                model.metabolites.get_by_id(row["identifier"]): row[
                    "new_coefficient"
                ]
            }
        )
    return reaction


biomass_W3110 = create_biomass(
    W3110_df_negative, W3110_df_positive, "W3110_biomass"
)

biomass_SA3 = create_biomass(
    SA3_df_negative, SA3_df_positive, "SA3_biomass"
)
# Show new reactions
display(biomass_W3110)
display(biomass_SA3)

# Add to model
model.add_reactions([biomass_W3110, biomass_SA3])

Reaction identifier,W3110_biomass
Name,
Memory address,0x07f30c3360a90
Stoichiometry,1.495426970760234 3pg_c + 0.10655682044443099 SHIKIMATE_c + 3.746364439181286 accoa_c + 59.78709032163742 atp_c + 0.3608617222222222 e4p_c + 0.07087284239766083 f6p_c + 0.12895058771929824 g3p_c +... 1.495426970760234 3-Phospho-D-glycerate + 0.10655682044443099 shikimate + 3.746364439181286 Acetyl-CoA + 59.78709032163742 ATP + 0.3608617222222222 D-Erythrose 4-phosphate + 0.07087284239766083...
GPR,
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,SA3_biomass
Name,
Memory address,0x07f30c32ac150
Stoichiometry,1.376890992366412 3pg_c + 22.148742610250682 SHIKIMATE_c + 3.4494064580152664 accoa_c + 55.0480282442748 atp_c + 0.33225778625954194 e4p_c + 0.06525506106870228 f6p_c + 0.11872923664122137 g3p_c +... 1.376890992366412 3-Phospho-D-glycerate + 22.148742610250682 shikimate + 3.4494064580152664 Acetyl-CoA + 55.0480282442748 ATP + 0.33225778625954194 D-Erythrose 4-phosphate + 0.06525506106870228...
GPR,
Lower bound,0.0
Upper bound,1000.0


## Obtaining solution fluxes

To visualize the fluxes for the pathway, we need to optimize our model. Using
COBRApy we run flux balance analysis twice and set for each run the respective
new biomass reaction as objective function. Using the COBRApy method
[cobra.model.optimize()](
  https://cobrapy.readthedocs.io/en/latest/autoapi/cobra/core/model/index.html#cobra.core.model.Model.optimize
) we get two [solutions objects](
  https://cobrapy.readthedocs.io/en/latest/autoapi/cobra/core/solution/index.html#cobra.core.solution.Solution
). 
We can see that the solution for the W3110 strain has a higher objective value
(0.2114) than the SA3 strain objective value (0.1058). Because shikimate has
a higher proportion in the biomass equation for SA3, this results a penalty
in the objective function.


In [11]:
# W3110 type
model.objective = "W3110_biomass"

solution_W3110 = model.optimize()
display(model.summary(solution=solution_W3110))

# SA3 strain
model.objective = "SA3_biomass"
solution_SA3 = model.optimize()
display(model.summary(solution=solution_SA3))

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,0.3776,1,0.63%
glc__D_e,EX_glc__D_e,10,6,99.37%
h2o_e,EX_h2o_e,7.088,0,0.00%
nh4_e,EX_nh4_e,1.152,0,0.00%
pi_e,EX_pi_e,0.7774,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-8.545,2,33.28%
etoh_e,EX_etoh_e,-8.243,2,32.10%
for_e,EX_for_e,-17.78,1,34.62%
h_e,EX_h_e,-30.54,0,0.00%


Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
h2o_e,EX_h2o_e,2.717,0,0.00%
nh4_e,EX_nh4_e,0.5313,0,0.00%
pi_e,EX_pi_e,0.3584,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-5.527,2,20.83%
co2_e,EX_co2_e,-2.448,1,4.61%
etoh_e,EX_etoh_e,-7.731,2,29.13%
for_e,EX_for_e,-13.62,1,25.67%
h_e,EX_h_e,-23.24,0,0.00%


## Visualization with Escher

The object [cobramod.Pathway](
module/cobramod/core/pathway/index.html#cobramod.core.pathway.Pathway
) has a method for visualizing metabolic pathway using [Escher](
https://escher.readthedocs.io/en/latest/
).
Escher is visualization tool capable to display metabolic maps. The function
[Pathway.visualize()](
module/cobramod/core/pathway/index.html#cobramod.core.pathway.Pathway.visualize
) generates a pathway map for the respective pathway. These pathway maps can be
easily customized to visualize flux distributions using default or user-defined 
colors and gradients.

In [12]:
model.groups.get_by_id("PWY-SHIKIMATE").visualize()

Builder(never_ask_before_quit=True, reaction_scale={}, reaction_styles=['color', 'text'])

To visually compare our results we set the colors of positive and
negative fluxes to blue and red, respectively. We set the map orientation to horizontal. To facilitate the comparison we set the bounds for the color 
gradient (`color_min_max`) to -1 and 1. The coloring of the arrows represents the values of the solution. Flux values near the bounds have a stronger
coloring, while flux value near 0 have a pale color. To visualize the solution,
we use the argument `solution_fluxes`.

We see that the first visualization displays the flux distribution for the
W3110 strain. In this example, the the values are very close to 0 and the
colors are very pale. The second visualization is the flux distribution for the SA3 strain. In contrast to the first map, the color blue and red are very 
strong because the flux values surpass the bounds (-1 , 1). Interestingly,
the reaction  `SHIKIMATE_5_DEHYDROGENASE_RXN_c` has in both visualization a
negative flux because the [reaction information](
  https://biocyc.org/ECOLI/NEW-IMAGE?type=REACTION&object=SHIKIMATE-5-DEHYDROGENASE-RXN
) shows the direction of the reaction from right to left. 

In [13]:
# Customize pathway
model.groups.get_by_id("PWY-SHIKIMATE").vertical = True
model.groups.get_by_id("PWY-SHIKIMATE").color_positive = "blue"
model.groups.get_by_id("PWY-SHIKIMATE").color_negative = "red"
model.groups.get_by_id("PWY-SHIKIMATE").color_min_max = [-1, 1]

# Visualize pathway with solutions
display(model.groups.get_by_id("PWY-SHIKIMATE").visualize(
    solution_fluxes=solution_W3110)
)
display(model.groups.get_by_id("PWY-SHIKIMATE").visualize(
    solution_fluxes=solution_SA3)
)

Builder(never_ask_before_quit=True, reaction_data={'ACALD': -8.242754606813811, 'ACALDt': 1.5948213258617017e-…

Builder(never_ask_before_quit=True, reaction_data={'ACALD': -7.731190549773717, 'ACALDt': 1.5948213258617017e-…

## References

1. Chen, Xianzhong, Mingming Li, Li Zhou, Wei Shen, Govender Algasan, You Fan, and Zhengxiang Wang. “Metabolic Engineering of Escherichia Coli for Improving Shikimate Synthesis from Glucose.” Bioresource Technology 166 (August 1, 2014): 64–71. https://doi.org/10.1016/j.biortech.2014.05.035.
2. Orth, Jeffrey D., R. M. T. Fleming, and Bernhard Ø. Palsson. “Reconstruction and Use of Microbial Metabolic Networks: The Core Escherichia Coli Metabolic Model as an Educational Guide.” EcoSal Plus 4, no. 1 (February 1, 2010). https://doi.org/10.1128/ecosalplus.10.2.1.
